# imports

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Patrickn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Patrickn\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Patrickn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Patrickn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [6]:
dataset_file_path = 'Dataset\Dataset.tsv'
dataset = pd.read_csv(dataset_file_path, sep='\t')

In [7]:
dataset

,label,text
0,Positive,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...
1,Positive,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...
2,Positive,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...
3,Positive,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...
4,Positive,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...
...,...,...
99994,Negative,معرفش ليه كنت عاوزة أكملها وهي مش عاجباني من ا...
99995,Negative,لا يستحق ان يكون في بوكنق لانه سيئ . لا شي. لا...
99996,Negative,كتاب ضعيف جدا ولم استمتع به. فى كل قصه سرد لحا...
99997,Negative,مملة جدا. محمد حسن علوان فنان بالكلمات، والوصف...


In [8]:
dataset['label'].value_counts()

label
Positive    33333
Mixed       33333
Negative    33333
Name: count, dtype: int64

# Add Id to Dataset

In [9]:
dataset['label_id'] = dataset['label'].map({'Positive' : 0,'Mixed' : 1,'Negative' : 2})

In [10]:
dataset

,label,text,label_id
0,Positive,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...,0
1,Positive,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...,0
2,Positive,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...,0
3,Positive,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...,0
4,Positive,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...,0
...,...,...,...
99994,Negative,معرفش ليه كنت عاوزة أكملها وهي مش عاجباني من ا...,2
99995,Negative,لا يستحق ان يكون في بوكنق لانه سيئ . لا شي. لا...,2
99996,Negative,كتاب ضعيف جدا ولم استمتع به. فى كل قصه سرد لحا...,2
99997,Negative,مملة جدا. محمد حسن علوان فنان بالكلمات، والوصف...,2


# Clean Dataset

In [11]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from __future__ import unicode_literals

In [12]:
arabic_stop_words = set(stopwords.words('arabic'))
lemmatizer = WordNetLemmatizer()

In [13]:
# 1.
def remove_stopwords(text):
  filtered_sentence = []
  text = re.sub(r'[^\w\s]', '', text)
  words = word_tokenize(text)
  for word in words:
      if word not in arabic_stop_words:
          lemmatized_word = lemmatizer.lemmatize(word)
          filtered_sentence.append(lemmatized_word)

  return filtered_sentence

In [14]:
# 2.
def remove_duplicate_char(text):
  new_sen = []
  for i in range(0, len(text)):
    new_word = re.sub(r'(.)\1+', r'\1', text[i])
    new_sen.append(new_word)

  return new_sen

In [15]:
# 3.
def remove_duplicate_words(text):
  new_sen = []
  for word in text:
    if word not in new_sen:
      new_sen.append(word)
    
  return new_sen

In [16]:
# 4.
import pyarabic.araby as araby
def remove_tashkeel(text):
  new_sen = []
  for i in range(0, len(text)):
    new_word = araby.normalize_hamza(text[i], method="tasheel")
    new_sen.append(new_word)
  return new_sen

In [17]:
# 5.
def remove_diacritics(text):
    new_sen = []
    for i in range(0, len(text)):
      arabic_diacritics = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
      new_word = re.sub(arabic_diacritics, '', text[i])
      new_sen.append(new_word)

    return text

In [18]:
# 6.
def preprocess_arabic_text(text):
    new_sen = []
    for i in range(0, len(text)):
      new_word = remove_diacritics(text[i])
      new_word = re.sub(r'\s+', ' ', text[i])
      new_word = re.sub(r'\d+', '', text[i])
      new_word = re.sub(r'[^\u0600-\u06FF\s]', '', text[i])  # Remove non-Arabic words
      new_word = re.sub(r'[،؛؟.!"#$%&\'()*+,-/:;<=>?@[\]^_`{|}~]', '', text[i])  # Remove punctuation
      new_sen.append(new_word)

    return new_sen

In [19]:
sen_list = dataset['text'].to_list()
cleaned_list = []
for sen in sen_list:
  remove_stopwords_sen = remove_stopwords(sen)
  remove_duplicate_char_sen = remove_duplicate_char(remove_stopwords_sen)
  remove_duplicate_words_sen = remove_duplicate_words(remove_duplicate_char_sen)
  remove_tashkeel_sen = remove_tashkeel(remove_duplicate_words_sen)
  remove_diacritics_sen = remove_diacritics(remove_tashkeel_sen)
  preprocess_arabic_sen = preprocess_arabic_text(remove_diacritics_sen)

  cleaned_list.append(preprocess_arabic_sen)

cleaned_list

[['متاز', 'نوعا', 'النظافة', 'والموقع', 'والتجهيز', 'والشاطيء', 'المطعم'],
 ['اسباب',
  'نجاح',
  'الامارات',
  'شخص',
  'الدولة',
  'يعشق',
  'ترابها',
  'نحب',
  'ومضات',
  'فكر',
  'نصايح',
  'لدولة',
  'تطمح',
  'بالصفوف',
  'الاولى',
  'قايد',
  'يقبل',
  'براحة',
  'شعبه',
  'وتوفر',
  'سب',
  'العيش',
  'الكريم',
  'حكم',
  'مواقف',
  'ونصايح',
  'لكل',
  'فرد',
  'فينا',
  'بمجرد',
  'كتاب',
  'سياسي',
  'كنت',
  'اعتقد',
  'يستحق',
  'القراءة',
  'مرات',
  'كثيرة'],
 ['هادفة',
  'وقوية',
  'تنقلك',
  'صخب',
  'شوارع',
  'القاهرة',
  'الى',
  'هدوء',
  'جبال',
  'الشيشان',
  'لتعرف',
  'حقيقة',
  'يجرى',
  'فى',
  'البلاد',
  'حروب',
  'ضاربة',
  'بحق',
  'المسلمين',
  'جزء',
  'كبير',
  'تاريخ',
  'المنطقة',
  'التضحية',
  'الرجولة',
  'الوفاء',
  'والكثير',
  'القيم',
  'الاخرى',
  'اثبت',
  'وجودها',
  'الرواية',
  'البسيطة'],
 ['خلصنا',
  'مبدييا',
  'الي',
  'مستني',
  'ابهار',
  'زي',
  'الفيل',
  'الازرق',
  'ميقراش',
  'احسن',
  'احمد',
  'مراد',
  'تخطى',
  'مرحلة',
  

In [20]:
dataset['cleaned_text'] = cleaned_list

In [21]:
dataset

,label,text,label_id,cleaned_text
0,Positive,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...,0,"[متاز, نوعا, النظافة, والموقع, والتجهيز, والشا..."
1,Positive,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...,0,"[اسباب, نجاح, الامارات, شخص, الدولة, يعشق, ترا..."
2,Positive,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...,0,"[هادفة, وقوية, تنقلك, صخب, شوارع, القاهرة, الى..."
3,Positive,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...,0,"[خلصنا, مبدييا, الي, مستني, ابهار, زي, الفيل, ..."
4,Positive,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...,0,"[ياسات, جلوريا, جزء, يتجزا, دبي, فندق, متكامل,..."
...,...,...,...,...
99994,Negative,معرفش ليه كنت عاوزة أكملها وهي مش عاجباني من ا...,2,"[معرفش, ليه, كنت, عاوزة, اكملها, وهي, مش, عاجب..."
99995,Negative,لا يستحق ان يكون في بوكنق لانه سيئ . لا شي. لا...,2,"[يستحق, ان, يكون, بوكنق, لانه, سيي, شي, يوجد, ..."
99996,Negative,كتاب ضعيف جدا ولم استمتع به. فى كل قصه سرد لحا...,2,"[كتاب, ضعيف, جدا, ولم, استمتع, فى, قصه, سرد, ل..."
99997,Negative,مملة جدا. محمد حسن علوان فنان بالكلمات، والوصف...,2,"[ملة, جدا, محمد, حسن, علوان, فنان, بالكلمات, و..."


# Aravec word embeddings

In [23]:
import gensim
import re
# import spacy
import numpy as np
from nltk import ngrams

from utilities import *

In [24]:
# download model from repo
model = gensim.models.Word2Vec.load("Models/full_grams_cbow_100_twitter/full_grams_cbow_100_twitter.mdl")
print("We've", len(model.wv.index_to_key),"vocabularies")

We've 1476715 vocabularies


In [7]:
%mkdir spacyModel

In [25]:
# export the word2vec fomart to the directory
model.wv.save_word2vec_format("./spacyModel/aravec.txt")

In [80]:
# !python -m spacy  init-model ar spacy.aravec.model --vectors-loc ./spacyModel/aravec.txt.gz

In [25]:
type(model)

gensim.models.word2vec.Word2Vec

In [26]:
model.wv['باتريك']

array([-0.6155143 ,  1.6617817 , -0.65107566,  0.7801767 , -0.44171965,
        0.61408705,  1.4658736 , -0.01211822,  0.28036454,  0.13424294,
       -0.04272187,  3.2989037 , -0.1983718 , -2.1049953 ,  0.34766114,
       -0.8656249 ,  0.69481516,  0.23421472, -1.612769  ,  0.6957242 ,
        0.64623624,  0.6797869 ,  0.6136975 ,  1.2302415 , -0.4203167 ,
       -1.017041  ,  0.5931764 , -1.4031352 ,  0.38363227,  0.9670074 ,
       -0.42372727,  0.1444707 ,  0.7598846 ,  0.22728498, -2.027577  ,
        1.1279352 ,  1.0537554 ,  2.6826155 ,  2.3630328 , -2.604296  ,
        0.02596986, -0.416042  ,  1.0014656 ,  0.12376305,  0.50148946,
        0.31688875,  0.96963716,  1.2682475 ,  1.0610051 , -1.1755881 ,
        0.6206213 , -0.8996107 ,  0.38458526,  0.9587003 ,  0.36197263,
       -0.46127725, -1.7254057 ,  2.3139265 ,  2.4134471 , -0.6908544 ,
       -0.5880119 , -0.6446584 ,  0.06795713, -0.28348657,  0.42798576,
       -2.9627788 , -2.7877235 ,  2.1008286 , -0.07585008,  1.91

In [27]:
model.wv['باتريك'].shape

(100,)

In [28]:
model.wv.most_similar('باتريك')

[('ويلسون', 0.809856116771698),
 ('اندريه', 0.7990731000900269),
 ('هاريس', 0.7686083912849426),
 ('فرانك', 0.7667828798294067),
 ('والتر', 0.7655714750289917),
 ('كريستيان', 0.7627326846122742),
 ('صامويل', 0.7617657780647278),
 ('هاريسون', 0.760664165019989),
 ('جايمس', 0.7591216564178467),
 ('انطونيو', 0.7570926547050476)]

In [29]:
model.wv.get_mean_vector('مرحبا انا باتريك').shape

(100,)

In [30]:
dataset['string_len'] = dataset['cleaned_text'].apply(len)


In [31]:
zero_count = dataset['string_len'].value_counts().get(0, 0)
zero_count

21

In [32]:
dataset

,label,text,label_id,cleaned_text,string_len
0,Positive,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...,0,"[متاز, نوعا, النظافة, والموقع, والتجهيز, والشا...",7
1,Positive,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...,0,"[اسباب, نجاح, الامارات, شخص, الدولة, يعشق, ترا...",38
2,Positive,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...,0,"[هادفة, وقوية, تنقلك, صخب, شوارع, القاهرة, الى...",33
3,Positive,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...,0,"[خلصنا, مبدييا, الي, مستني, ابهار, زي, الفيل, ...",77
4,Positive,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...,0,"[ياسات, جلوريا, جزء, يتجزا, دبي, فندق, متكامل,...",11
...,...,...,...,...,...
99994,Negative,معرفش ليه كنت عاوزة أكملها وهي مش عاجباني من ا...,2,"[معرفش, ليه, كنت, عاوزة, اكملها, وهي, مش, عاجب...",25
99995,Negative,لا يستحق ان يكون في بوكنق لانه سيئ . لا شي. لا...,2,"[يستحق, ان, يكون, بوكنق, لانه, سيي, شي, يوجد, ...",14
99996,Negative,كتاب ضعيف جدا ولم استمتع به. فى كل قصه سرد لحا...,2,"[كتاب, ضعيف, جدا, ولم, استمتع, فى, قصه, سرد, ل...",13
99997,Negative,مملة جدا. محمد حسن علوان فنان بالكلمات، والوصف...,2,"[ملة, جدا, محمد, حسن, علوان, فنان, بالكلمات, و...",24


In [33]:
dataset = dataset[dataset['string_len'] != 0]

In [34]:
dataset

,label,text,label_id,cleaned_text,string_len
0,Positive,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...,0,"[متاز, نوعا, النظافة, والموقع, والتجهيز, والشا...",7
1,Positive,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...,0,"[اسباب, نجاح, الامارات, شخص, الدولة, يعشق, ترا...",38
2,Positive,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...,0,"[هادفة, وقوية, تنقلك, صخب, شوارع, القاهرة, الى...",33
3,Positive,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...,0,"[خلصنا, مبدييا, الي, مستني, ابهار, زي, الفيل, ...",77
4,Positive,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...,0,"[ياسات, جلوريا, جزء, يتجزا, دبي, فندق, متكامل,...",11
...,...,...,...,...,...
99994,Negative,معرفش ليه كنت عاوزة أكملها وهي مش عاجباني من ا...,2,"[معرفش, ليه, كنت, عاوزة, اكملها, وهي, مش, عاجب...",25
99995,Negative,لا يستحق ان يكون في بوكنق لانه سيئ . لا شي. لا...,2,"[يستحق, ان, يكون, بوكنق, لانه, سيي, شي, يوجد, ...",14
99996,Negative,كتاب ضعيف جدا ولم استمتع به. فى كل قصه سرد لحا...,2,"[كتاب, ضعيف, جدا, ولم, استمتع, فى, قصه, سرد, ل...",13
99997,Negative,مملة جدا. محمد حسن علوان فنان بالكلمات، والوصف...,2,"[ملة, جدا, محمد, حسن, علوان, فنان, بالكلمات, و...",24


In [35]:
dataset['cleaned_text_token'] = dataset['cleaned_text'].apply(model.wv.get_mean_vector)

C:\Users\Patrickn\AppData\Local\Temp\ipykernel_11356\3424571786.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['cleaned_text_token'] = dataset['cleaned_text'].apply(model.wv.get_mean_vector)


In [36]:
dataset

,label,text,label_id,cleaned_text,string_len,cleaned_text_token
0,Positive,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...,0,"[متاز, نوعا, النظافة, والموقع, والتجهيز, والشا...",7,"[-0.048139643, -0.051170737, 0.057719726, 0.01..."
1,Positive,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...,0,"[اسباب, نجاح, الامارات, شخص, الدولة, يعشق, ترا...",38,"[-0.02937056, 0.05526421, 0.02213953, 0.032197..."
2,Positive,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...,0,"[هادفة, وقوية, تنقلك, صخب, شوارع, القاهرة, الى...",33,"[-0.017106792, 0.07227743, 0.04705416, 0.04526..."
3,Positive,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...,0,"[خلصنا, مبدييا, الي, مستني, ابهار, زي, الفيل, ...",77,"[-0.0065515684, 0.046778645, 0.012682784, 0.07..."
4,Positive,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...,0,"[ياسات, جلوريا, جزء, يتجزا, دبي, فندق, متكامل,...",11,"[-0.005888562, -0.006276867, 0.061604027, 0.05..."
...,...,...,...,...,...,...
99994,Negative,معرفش ليه كنت عاوزة أكملها وهي مش عاجباني من ا...,2,"[معرفش, ليه, كنت, عاوزة, اكملها, وهي, مش, عاجب...",25,"[-0.00030626432, 0.02978748, -0.020429999, 0.0..."
99995,Negative,لا يستحق ان يكون في بوكنق لانه سيئ . لا شي. لا...,2,"[يستحق, ان, يكون, بوكنق, لانه, سيي, شي, يوجد, ...",14,"[-0.044104204, -0.001613312, 0.032425657, -0.0..."
99996,Negative,كتاب ضعيف جدا ولم استمتع به. فى كل قصه سرد لحا...,2,"[كتاب, ضعيف, جدا, ولم, استمتع, فى, قصه, سرد, ل...",13,"[-0.068677664, 0.11327193, 0.013488528, -0.004..."
99997,Negative,مملة جدا. محمد حسن علوان فنان بالكلمات، والوصف...,2,"[ملة, جدا, محمد, حسن, علوان, فنان, بالكلمات, و...",24,"[-0.036897708, 0.05546034, 0.03913802, 0.00126..."


# Training

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    dataset['cleaned_text_token'].values,
    dataset['label_id'],
    test_size=0.1,
    stratify=dataset['label_id']
    )

In [38]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((89980,), (9998,), (89980,), (9998,))

In [42]:
import tensorflow as tf
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

X_train = np.stack(X_train)
X_test = np.stack(X_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

dl_model = Sequential()

dl_model.add(Dense(512, input_shape=(100,)))
dl_model.add(Activation('tanh'))

#dl_model.add(Dropout(0.5))
dl_model.add(Dense(256))
dl_model.add(Activation('tanh'))
dl_model.add(Dense(128))
dl_model.add(Activation('tanh'))
dl_model.add(Dense(64))
dl_model.add(Activation('tanh'))
dl_model.add(Dense(32))
dl_model.add(Activation('tanh'))
dl_model.add(Dense(3))
dl_model.add(Activation('softmax'))

dl_model.compile(
    loss='SparseCategoricalCrossentropy',
    optimizer='adam',
    metrics=['accuracy']
    )


dl_model.fit(X_train, y_train,batch_size=64,epochs=200)

Epoch 1/200
1406/1406 [==============================] - 5s 3ms/step - loss: 0.9465 - accuracy: 0.5397
Epoch 2/200
1406/1406 [==============================] - 5s 3ms/step - loss: 0.9028 - accuracy: 0.5647
Epoch 3/200
1406/1406 [==============================] - 4s 3ms/step - loss: 0.8874 - accuracy: 0.5713
Epoch 4/200
1406/1406 [==============================] - 4s 3ms/step - loss: 0.8780 - accuracy: 0.5784
Epoch 5/200
1406/1406 [==============================] - 4s 3ms/step - loss: 0.8719 - accuracy: 0.5823
Epoch 6/200
1406/1406 [==============================] - 4s 3ms/step - loss: 0.8662 - accuracy: 0.5844
Epoch 7/200
1406/1406 [==============================] - 4s 3ms/step - loss: 0.8611 - accuracy: 0.5878
Epoch 8/200
1406/1406 [==============================] - 4s 3ms/step - loss: 0.8566 - accuracy: 0.5906
Epoch 9/200
1406/1406 [==============================] - 4s 3ms/step - loss: 0.8519 - accuracy: 0.5932
Epoch 10/200
1406/1406 [==============================] - 4s 3ms/step - l